In [53]:
# === Load new clean OLS base ===
base = pd.read_csv("../results/ols_predictions_all.csv", skipinitialspace=True)
base["yyyymm"] = pd.to_datetime(base["yyyymm"], errors="coerce")
base = base.dropna(subset=["yyyymm"])
base["year"] = base["yyyymm"].dt.year.astype(int)
base["month"] = base["yyyymm"].dt.month.astype(int)
base["permno"] = base["permno"].astype(int)
base = base.drop_duplicates(subset=["permno", "year", "month"])
base = base[["permno", "year", "month", "yyyymm", "stock_exret", "ols"]]

# === Model files to merge (with column renaming) ===
model_files = {
    "ols_reduced": ("../results/ols_predictions_reduced.csv", "ols"),
    "lasso": ("../results/lasso_predictions_all.csv", "lasso"),
    "ridge": ("../results/ridge_predictions_reduced.csv", "ridge"),
    "enet": ("../results/enet_predictions_reduced_simple.csv", "enet_simple"),
    "xgb": ("../results/xgb_predictions_reduced.csv", "xgb"),
    "ae_ridge": ("../results/ae_pred_ridgescaled.csv", "ae_ridge"),
}

# === Merge all with cleaning using skipinitialspace ===
for name, (path, pred_col) in model_files.items():
    df = pd.read_csv(path, skipinitialspace=True)
    df["yyyymm"] = pd.to_datetime(df["yyyymm"], errors="coerce")
    df = df.dropna(subset=["yyyymm"])
    df["year"] = df["yyyymm"].dt.year.astype(int)
    df["month"] = df["yyyymm"].dt.month.astype(int)
    df["permno"] = df["permno"].astype(int)
    df = df.rename(columns={pred_col: name})
    df = df[["permno", "year", "month", name]]

    base = pd.merge(base, df, on=["permno", "year", "month"], how="inner")

In [54]:
base.head(2)

,permno,year,month,yyyymm,stock_exret,ols,ols_reduced,lasso,ridge,enet,xgb,ae_ridge
0,10104,2010,1,2010-01-01,-0.057888,-0.004766,0.002876,0.001217,0.000195,-0.000503,-0.004590,-0.012371
1,10107,2010,1,2010-01-01,-0.075459,0.010014,0.007736,0.003348,0.009430,0.004289,-0.005139,0.000072


In [55]:
# Load surprise prediction file and merge using left join on permno + year + month

# Step 1: Load and prepare surprise_predictions.csv
surprise_df = pd.read_csv("../results/surprise_predictions_updated.csv", skipinitialspace=True)
surprise_df["date"] = pd.to_datetime(surprise_df["date"], errors="coerce")
surprise_df = surprise_df.dropna(subset=["date"])
surprise_df["year"] = surprise_df["date"].dt.year.astype(int)
surprise_df["month"] = surprise_df["date"].dt.month.astype(int)
surprise_df["permno"] = surprise_df["permno"].astype(int)

# Keep only necessary columns
surprise_df = surprise_df[["permno", "year", "month", "pred_surprise"]]

# Step 2: Merge with existing base table using left join
base_with_surprise = pd.merge(base, surprise_df, on=["permno", "year", "month"], how="left")

In [56]:
base_with_surprise.head(2)

,permno,year,month,yyyymm,stock_exret,ols,ols_reduced,lasso,ridge,enet,xgb,ae_ridge,pred_surprise
0,10104,2010,1,2010-01-01,-0.057888,-0.004766,0.002876,0.001217,0.000195,-0.000503,-0.004590,-0.012371,0.010672
1,10107,2010,1,2010-01-01,-0.075459,0.010014,0.007736,0.003348,0.009430,0.004289,-0.005139,0.000072,0.015960


In [57]:
base_with_surprise.to_csv('../results/prediction_results.csv', index=False)